# TAREA DE EJERCICIO

Actualiza el primer proyecto para resumir una página web y utilizar un modelo de código abierto que se ejecute localmente a través de Ollama en lugar de OpenAI

Podrás utilizar esta técnica para todos los proyectos posteriores si prefiere no utilizar API de pago (closed source).

**Beneficios:**
1. Sin cargos por API: código abierto
2. Los datos no salen de su ordenador

**Desventajas:**
1. Tiene mucha menos potencia (parámetros) que el modelo frontera

## Resumen de la instalación de Ollama

¡Simplemente visita [ollama.com](https://ollama.com) e instálalo!

Una vez que hayas terminado, el servidor ollama ya debería estar ejecutándose localmente.
Si entras en:
[http://localhost:11434/](http://localhost:11434/)

Debería ver el mensaje `Ollama se está ejecutando`.

De lo contrario, abre una nueva Terminal (Mac) o Powershell (Windows) e introduce `ollama serve`.
Luego, intenta entrar em [http://localhost:11434/](http://localhost:11434/) nuevamente.

In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constantes

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Crea una lista de mensajes utilizando el mismo formato que usamos para OpenAI

messages = [
    {"role": "user", "content": "Describe algunas de las aplicaciones comerciales de la IA generativa."}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

La Inteligencia Artificial (IA) generativa es una rama de la IA que se enfoca en crear contenido original a partir de datos y patrones existentes, como imágenes, textos y audio. Algunas de las aplicaciones comerciales más comunes de la IA generativa incluyen:

1. **Generación de contenido**: La IA generativa puede ser utilizada para crear contenido automático, como artículos de blog, descripciones de productos y publicaciones en redes sociales.
2. **Diseño gráfico y visualización**: La IA generativa puede ayudar a diseñadores a crear diseños gráficos más efectivos y a visualizar datos complejos de manera más sencilla.
3. **Creación de música y audio**: La IA generativa puede ser utilizada para crear música original y sonidos de efecto que pueden ser utilizados en películas, videojuegos y otros medios de comunicación.
4. **Diseño de productos**: La IA generativa puede ayudar a diseñadores a crear diseños de productos más innovadores y efectivos.
5. **Generación de personajes y objetos v

# Presentación del paquete ollama

Ahora haremos lo mismo, pero utilizando el elegante paquete de Python ollama en lugar de una llamada HTTP directa.

En esencia, se realiza la misma llamada que se indicó anteriormente al servidor ollama que se ejecuta en localhost:11434

In [7]:
import ollama
from IPython.display import Markdown, display

response = ollama.chat(model=MODEL, messages=messages)
display(Markdown(response['message']['content']))

La inteligencia artificial generativa (IAG) se refiere al uso de técnicas de aprendizaje automático y modelos de red neuronal para generar contenido nuevo, imitando a los seres humanos. Algunas de las aplicaciones comerciales más comunes de la IAG son:

1. **Generación de contenido**: La IAG puede ser utilizada para generar contenido como texto, imágenes, videos y audio, lo que puede ser utilizado en marketing, publicidad y comunicación.
 * Ejemplos: Generar artículos de blog, crear contenido social, diseñar anuncios.
2. **Diseño asistido por inteligencia artificial**: La IAG puede ayudar a los diseñadores a generar ideas y diseños nuevos, lo que puede ser utilizado en la creación de productos, arquitectura y moda.
 * Ejemplos: Generar diseños de ropa, crear patrones para textiles, diseñar edificios.
3. **Generación de música y audio**: La IAG puede ser utilizada para generar música y audio de alta calidad, lo que puede ser utilizado en la creación de contenido musical y en la industria del entretenimiento.
 * Ejemplos: Generar canciones, crear efectos sonoros, diseñar sonidos para películas y videojuegos.
4. **Generación de textos y traducciones**: La IAG puede ser utilizada para generar textos y traducir texto de un idioma a otro, lo que puede ser utilizado en la creación de contenido y la comunicación internacional.
 * Ejemplos: Generar resúmenes de artículos, traducir texto de inglés a español, crear contenido para sitios web internacionales.
5. **Generación de imágenes y videos**: La IAG puede ser utilizada para generar imágenes y videos de alta calidad, lo que puede ser utilizado en la creación de contenido visual y en la industria del entretenimiento.
 * Ejemplos: Generar imágenes de productores, crear efectos visuales para películas y videojuegos, diseñar anuncios.
6. **Análisis de datos y predicción**: La IAG puede ser utilizada para analizar grandes cantidades de datos y generar predicciones sobre patrones y tendencias, lo que puede ser utilizado en la toma de decisiones empresariales y en la industria financiera.
 * Ejemplos: Analizar datos de ventas, predecir patrones de comportamiento, diseñar modelos de crecimiento para empresas.

En resumen, la IAG tiene un amplio rango de aplicaciones comerciales que pueden ser utilizadas para generar contenido, mejorar la eficiencia y la productividad, y crear nuevos productos y servicios.

# AHORA el ejercicio para ti

Toma el código notebook 01 e incorpóralo aquí para crear un resumidor de sitios web que use Llama 3.2 ejecutándose localmente en lugar de OpenAI

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service("chromedriver-mac-arm64/chromedriver")
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [9]:
system_prompt = """ Eres un asistente que analiza el contenido de un sitio web
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación.
Responder en Markdown."""

In [10]:
def user_prompt_for(website):
    title = getattr(website, "title", None)
    if title is None and isinstance(website, dict):
        title = website.get("title", "")
    text = getattr(website, "text", None)
    if text is None and isinstance(website, dict):
        text = website.get("content", "")
    user_prompt = f"""
    Estás viendo un sitio web titulado {title}.
    El contenido de este sitio web es el siguiente: {text}.
    Si incluye noticias, productos o anuncios, resúmelos también.
    """
    return user_prompt

In [11]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [12]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response['message']['content']

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://marca.com")

El texto proporcionado no es un artículo de noticias ni un contenido específico de la página MARCA, sino más bien una sección de resultados y clasificaciones del fútbol español y europeo, así como noticias relacionadas con el deporte. No contienen información sobre un evento específico o contenido relevante que se pueda resumir o analizar.